In [75]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import ast
import warnings
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [77]:
import pandas as pd
data = pd.read_csv('/home/asus/New_Project_Churn/data/Dataset.csv')


In [78]:
data = pd.read_csv('/home/asus/New_Project_Churn/data/Dataset.csv')
data.head(5)

,CustomerID,Name,Age,Gender,Location,Email,Phone,Address,Segment,PurchaseHistory,SubscriptionDetails,ServiceInteractions,PaymentHistory,WebsiteUsage,ClickstreamData,EngagementMetrics,Feedback,MarketingCommunication,NPS,ChurnLabel,Timestamp
0,1001,Mark Barrett,31,Male,Andrewfort,allison74@example.net,3192528777,"61234 Shelley Heights Suite 467\nCohentown, GU...",Segment B,"[{'Product': 'Frozen Cocktail Mixes', 'Frequen...","{'Plan': 'Express', 'Start_Date': '2020-06-08'...","[{'Type': 'Call', 'Date': '2019-09-26'}, {'Typ...","[{'Method': 'Credit Card', 'Late_Payments': 5}...","{'PageViews': 49, 'TimeSpent(minutes)': 15}","[{'Action': 'Add to Cart', 'Page': 'register',...","{'Logins': 19, 'Frequency': 'Weekly'}","{'Rating': 1, 'Comment': 'I move baby go small...","[{'Email_Sent': '2019-10-17', 'Email_Opened': ...",3,1,2020-01-27 01:36:49
1,1002,Jeremy Welch,66,Female,Millerhaven,fmiller@example.com,231-587-1818x8651,"4959 Jennifer Junction\nNew Angelaport, TN 87397",Segment C,"[{'Product': 'Watercraft Polishes', 'Frequency...","{'Plan': 'Pro', 'Start_Date': '2021-07-21', 'E...","[{'Type': 'Call', 'Date': '2020-01-05'}, {'Typ...","[{'Method': 'Credit Card', 'Late_Payments': 3}...","{'PageViews': 100, 'TimeSpent(minutes)': 9}","[{'Action': 'Add to Cart', 'Page': 'homepage',...","{'Logins': 9, 'Frequency': 'Weekly'}","{'Rating': 2, 'Comment': 'Wish what bag cut li...","[{'Email_Sent': '2021-08-02', 'Email_Opened': ...",6,0,2019-01-06 18:30:03
2,1003,Brandon Patel,36,Female,Lozanostad,jasonbrown@example.org,(270)633-9095,"38701 Amanda Brook Apt. 076\nKimshire, NJ 62516",Segment B,"[{'Product': 'Vehicle Waxes, Polishes & Protec...","{'Plan': 'Essential', 'Start_Date': '2019-10-0...","[{'Type': 'Email', 'Date': '2019-10-09'}, {'Ty...","[{'Method': 'Credit Card', 'Late_Payments': 1}...","{'PageViews': 1, 'TimeSpent(minutes)': 97}","[{'Action': 'Search', 'Page': 'terms', 'Timest...","{'Logins': 19, 'Frequency': 'Monthly'}","{'Rating': 4, 'Comment': 'Some Democrat guess ...","[{'Email_Sent': '2021-08-29', 'Email_Opened': ...",3,0,2019-04-30 04:25:10
3,1004,Tina Martin,62,Female,South Dustin,matthew62@example.net,050.081.8706x11982,"67324 Ashley Coves\nSouth John, RI 29650",Segment C,"[{'Product': 'Mouthwash', 'Frequency': 5, 'Val...","{'Plan': 'Smart', 'Start_Date': '2020-01-14', ...","[{'Type': 'Call', 'Date': '2020-08-28'}, {'Typ...","[{'Method': 'Credit Card', 'Late_Payments': 36...","{'PageViews': 25, 'TimeSpent(minutes)': 31}","[{'Action': 'Click', 'Page': 'privacy', 'Times...","{'Logins': 4, 'Frequency': 'Daily'}","{'Rating': 1, 'Comment': 'Yard feel never miss...","[{'Email_Sent': '2021-02-03', 'Email_Opened': ...",1,1,2020-03-03 17:33:28
4,1005,Christopher Rodriguez,68,Female,West James,shannonstrickland@example.org,+1-701-854-4915x724,"01169 Miller Mission\nWest Anthonyburgh, WY 47359",Segment C,"[{'Product': 'Ice Cream Novelties', 'Frequency...","{'Plan': 'Basic', 'Start_Date': '2021-04-08', ...","[{'Type': 'Call', 'Date': '2019-04-10'}, {'Typ...","[{'Method': 'Credit Card', 'Late_Payments': 0}...","{'PageViews': 77, 'TimeSpent(minutes)': 51}","[{'Action': 'Click', 'Page': 'privacy', 'Times...","{'Logins': 12, 'Frequency': 'Weekly'}","{'Rating': 3, 'Comment': 'Ten determine unit i...","[{'Email_Sent': '2022-03-11', 'Email_Opened': ...",3,0,2019-04-05 22:42:22


In [79]:
def normalize(df: pd.DataFrame, fk: str, column:str) -> pd.DataFrame:
    """
    This function is expected to extract the specified columns as Dataframe
    """
    normalized_df = pd.DataFrame({
        fk: df[fk],
        column : df[column]
    }) 

    normalized_df[column] = normalized_df[column].apply(ast.literal_eval)
    
    if isinstance(normalized_df[column][0], list):
        normalized_df = normalized_df.explode(column)

    normalized_df = pd.concat([
        normalized_df.drop(columns=[column]),
        normalized_df[column].apply(pd.Series)
    ], axis =  1)

    return normalized_df

In [80]:
normalize_table = data.iloc[:, 9:18].columns.to_list()

normalize_columns = {}

for column in normalize_table:
    normalize_columns[column] = normalize(data, 'CustomerID', column)

In [81]:
data.drop(columns = normalize_table, axis = 1, inplace = True)

In [82]:
 # aggregate the normalized data for PurchaseHistory
purchaseHistory = normalize_columns['PurchaseHistory'].groupby('CustomerID').agg(
    TotalPurchaseFrequency=('Frequency', lambda x: x.sum()),
    TotalPurchaseValue=('Value', lambda x: x.sum()),
    ProductList=('Product', lambda x: '|'.join(sorted(set(x))))
).reset_index()

# subscription details
subscriptionDetails = normalize_columns['SubscriptionDetails']

# aggregate the normalized data for service interactions
serviceInteractions = normalize_columns['ServiceInteractions'].groupby('CustomerID').agg(
    TotalInteractionType=('Type', lambda x: '|'.join(sorted(set(x)))),
    numEmails=('Type', lambda x: (x == 'Email').sum()),
    numCalls=('Type', lambda x: (x == 'Call').sum()),
    numChats=('Type', lambda x: (x == 'Chat').sum()),
    FirstInteractionDate=('Date', 'min'),
    LastInteractionDate=('Date', 'max')
).reset_index()

# aggregate for payment history
paymentHistory = normalize_columns['PaymentHistory'].groupby('CustomerID').agg(
    AVGLatePayment=('Late_Payments', 'mean'),
    NumPaymentMethods=('Method', 'nunique'),
).reset_index()

# website usage
websiteUsage = normalize_columns['WebsiteUsage']

# aggregate for clickstream data
clickstreamData = normalize_columns['ClickstreamData'].groupby('CustomerID').agg(
    ActionCount=('Action', 'count'),
    unique_pages=('Page', pd.Series.nunique),
    most_recent_action_date=('Timestamp', 'max')    
).reset_index()

# engagement Metrics
engagementMetrics = normalize_columns['EngagementMetrics']

# feedback
feedback = normalize_columns['Feedback']

# marketing Communications
df_mc = normalize_columns['MarketingCommunication'].copy()

# Convert columns to datetime
df_mc['Email_Sent'] = pd.to_datetime(df_mc['Email_Sent'])
df_mc['Email_Opened'] = pd.to_datetime(df_mc['Email_Opened'])
df_mc['Email_Clicked'] = pd.to_datetime(df_mc['Email_Clicked'])

# Calculate per-row differences in days
df_mc['OpenDays'] = (df_mc['Email_Opened'] - df_mc['Email_Sent']).dt.days
df_mc['ClickDays'] = (df_mc['Email_Clicked'] - df_mc['Email_Opened']).dt.days

# Now groupby and aggregate mean of these differences
marketingCommunications = df_mc.groupby('CustomerID').agg(
    AVGOpenDays = ('OpenDays', 'mean'),
    AVGClickDays = ('ClickDays', 'mean')
).reset_index()

In [83]:
# Start with the main data copy
merged_data = data.copy()

# List of aggregated DataFrames to merge
dfs_to_merge = [
    purchaseHistory,
    subscriptionDetails,
    serviceInteractions,
    paymentHistory,
    websiteUsage,
    clickstreamData,
    engagementMetrics,
    feedback,
    marketingCommunications
]

for df_to_merge in dfs_to_merge:
    merged_data = merged_data.merge(df_to_merge, on='CustomerID', how='left')

In [84]:
merged_data

,CustomerID,Name,Age,Gender,Location,Email,Phone,Address,Segment,NPS,ChurnLabel,Timestamp,TotalPurchaseFrequency,TotalPurchaseValue,ProductList,Plan,Start_Date,End_Date,TotalInteractionType,numEmails,numCalls,numChats,FirstInteractionDate,LastInteractionDate,AVGLatePayment,NumPaymentMethods,PageViews,TimeSpent(minutes),ActionCount,unique_pages,most_recent_action_date,Logins,Frequency,Rating,Comment,AVGOpenDays,AVGClickDays
0,1001,Mark Barrett,31,Male,Andrewfort,allison74@example.net,3192528777,"61234 Shelley Heights Suite 467\nCohentown, GU...",Segment B,3,1,2020-01-27 01:36:49,38,3994.72,Frozen Cocktail Mixes|Guacamole|Hockey Stick C...,Express,2020-06-08,2022-10-27,Call|Chat|Email,1,1,2,2019-09-26,2021-07-25,13.333333,3,49,15,24,13,2022-11-07 02:24:31,19,Weekly,1,I move baby go small big. Office institution s...,818.0,319.0
1,1002,Jeremy Welch,66,Female,Millerhaven,fmiller@example.com,231-587-1818x8651,"4959 Jennifer Junction\nNew Angelaport, TN 87397",Segment C,6,0,2019-01-06 18:30:03,4,2844.35,Baby Protective Wear|Footbags|Watercraft Polishes,Pro,2021-07-21,2022-05-07,Call|Chat|Email,10,5,4,2019-01-12,2022-12-13,3.333333,3,100,9,24,13,2022-12-05 00:27:08,9,Weekly,2,Wish what bag cut life. Statement might opport...,110.0,88.0
2,1003,Brandon Patel,36,Female,Lozanostad,jasonbrown@example.org,(270)633-9095,"38701 Amanda Brook Apt. 076\nKimshire, NJ 62516",Segment B,3,0,2019-04-30 04:25:10,14,1866.52,"Fudge|Pipe Caps & Plugs|Vehicle Waxes, Polishe...",Essential,2019-10-05,2020-08-19,Call|Chat|Email,1,1,1,2019-10-09,2022-01-04,2.666667,3,1,97,12,7,2022-11-02 15:59:43,19,Monthly,4,Some Democrat guess but short. Whether behind ...,333.0,117.0
3,1004,Tina Martin,62,Female,South Dustin,matthew62@example.net,050.081.8706x11982,"67324 Ashley Coves\nSouth John, RI 29650",Segment C,1,1,2020-03-03 17:33:28,28,1378.64,Crêpe & Blini Pans|Electrical Switches|Mouthwa...,Smart,2020-01-14,2022-03-27,Call|Chat|Email,18,17,24,2019-01-03,2022-11-10,26.333333,3,25,31,47,14,2022-12-08 18:10:18,4,Daily,1,Yard feel never miss ask billion Congress. Fly...,159.0,423.0
4,1005,Christopher Rodriguez,68,Female,West James,shannonstrickland@example.org,+1-701-854-4915x724,"01169 Miller Mission\nWest Anthonyburgh, WY 47359",Segment C,3,0,2019-04-05 22:42:22,39,2425.05,Audiobooks|Fire Extinguisher & Equipment Stora...,Basic,2021-04-08,2022-11-09,Call|Chat|Email,5,4,1,2019-04-10,2022-12-19,0.666667,3,77,51,30,12,2022-12-30 23:37:00,12,Weekly,3,Ten determine unit interview challenge stock. ...,193.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,13479,Catherine Lee,55,Female,New Lauren,kgarcia@example.com,(977)150-1569x426,"836 Gonzalez Drive Apt. 334\nLake Kelly, RI 01667",Segment A,8,0,2022-01-27 00:27:28,14,1196.56,Concrete & Masonry Cleaners|Ice Cream Noveltie...,Essential,2019-06-15,2021-06-29,Call|Chat|Email,3,3,4,2019-05-07,2022-10-09,1.000000,3,70,57,6,6,2022-10-02 14:18:14,22,Daily,2,Light appear fight lawyer where star.,126.0,443.0
12479,13480,Kimberly Johnson,29,Male,Randymouth,carrjennifer@example.org,485.014.7824x5235,3715 Wiggins Harbors Suite 139\nEast Gregorybo...,Segment A,7,0,2020-06-14 23:04:47,1,710.57,Straight Pins,Flex,2022-12-10,2022-12-28,Call|Chat,0,1,2,2019-12-11,2022-11-05,2.000000,3,71,66,9,8,2022-11-10 14:12:37,25,Weekly,3,Yet very girl history. Thing late dream you re...,167.0,40.0
12480,13481,Lisa Rodgers,38,Male,East Erin,william37@example.com,951-489-9554,"56050 Steven Summit Suite 383\nJohnmouth, NM 2...",Segment C,1,1,2020-10-13 15:09:13,63,5154.42,Cabbage|Fabric Refreshers|Furisode Kimonos|Gol...,Deluxe,2021-07-04,2021-07-24,Call|Chat|Email,11,9,6,2019-01-12,2022-12-08,27.666667,3,96,1,26,9,2022-11-20 23:26:42,9,Monthly,5,Offer particularly single degree seem sound. S...,383.0,459.0
12481,13482,Kristin Carey,26,Female,New Leah,josephcarter@example.net,001-335-464-3236x722,"72356 Teresa Rapid\nPorterborough, SC 40

# RFM Analysis

In [85]:
merged_data.columns

Index(['CustomerID', 'Name', 'Age', 'Gender', 'Location', 'Email', 'Phone',
       'Address', 'Segment', 'NPS', 'ChurnLabel', 'Timestamp',
       'TotalPurchaseFrequency', 'TotalPurchaseValue', 'ProductList', 'Plan',
       'Start_Date', 'End_Date', 'TotalInteractionType', 'numEmails',
       'numCalls', 'numChats', 'FirstInteractionDate', 'LastInteractionDate',
       'AVGLatePayment', 'NumPaymentMethods', 'PageViews',
       'TimeSpent(minutes)', 'ActionCount', 'unique_pages',
       'most_recent_action_date', 'Logins', 'Frequency', 'Rating', 'Comment',
       'AVGOpenDays', 'AVGClickDays'],
      dtype='object')

In [86]:
# merged_data['LastInteractionDate']

merged_data['LastInteractionDate'] = pd.to_datetime(merged_data['LastInteractionDate'], errors = 'coerce')

In [87]:
currentDate = merged_data['LastInteractionDate'].max()

merged_data['recency'] = currentDate - merged_data['LastInteractionDate']
merged_data['frequency'] = merged_data['TotalPurchaseFrequency'] * merged_data['Logins']
merged_data['monetary'] = merged_data['TotalPurchaseValue']

r_label = range(5, 0, -1)
f_label = range(1, 6)
m_label = range(1, 6)


merged_data['recency'] = pd.qcut(merged_data['recency'], q=5, labels = r_label)
merged_data['frequency'] = pd.qcut(merged_data['frequency'], q=5, labels = r_label)
merged_data['monetary'] = pd.qcut(merged_data['monetary'], q=5, labels = r_label)

merged_data[['frequency', 'recency', 'monetary']] = merged_data[['frequency', 'recency', 'monetary']].astype('int32')
merged_data['rfm_value'] = merged_data['recency'] + merged_data['frequency'] + merged_data['monetary']

bins = [3, 6, 9, 12, 15]
labels = ['at_risk', 'need_attention', 'loyal', 'premium']

merged_data['customer_segment'] = pd.cut(merged_data['rfm_value'], bins = bins, labels = labels)
merged_data.drop(columns = ['recency', 'frequency', 'monetary', 'rfm_value'], inplace = True, axis = 1)

In [88]:
merged_data['customer_segment'].value_counts()

customer_segment
need_attention    4350
loyal             3927
at_risk           2388
premium           1590
Name: count, dtype: int64

In [89]:
merged_data.to_csv('EDA_Cleaned_Data.csv', index = False)

# Feature Encoding

In [90]:
merged_data.dtypes

CustomerID                          int64
Name                               object
Age                                 int64
Gender                             object
Location                           object
Email                              object
Phone                              object
Address                            object
Segment                            object
NPS                                 int64
ChurnLabel                          int64
Timestamp                          object
TotalPurchaseFrequency              int64
TotalPurchaseValue                float64
ProductList                        object
Plan                               object
Start_Date                         object
End_Date                           object
TotalInteractionType               object
numEmails                           int64
numCalls                            int64
numChats                            int64
FirstInteractionDate               object
LastInteractionDate        datetim

In [91]:
# merged_data.drop(columns = ['Name', 'Email', 'Address', 'Phone', 'Comment'], axis = 1, inplace = True)
merged_data.select_dtypes(include = 'O').dtypes

Name                       object
Gender                     object
Location                   object
Email                      object
Phone                      object
Address                    object
Segment                    object
Timestamp                  object
ProductList                object
Plan                       object
Start_Date                 object
End_Date                   object
TotalInteractionType       object
FirstInteractionDate       object
most_recent_action_date    object
Frequency                  object
Comment                    object
dtype: object

In [92]:
date_cols = merged_data[['Start_Date', 'End_Date', 'Timestamp', 'FirstInteractionDate', 'most_recent_action_date']].columns.to_list()
for col in date_cols:
    merged_data[col] = pd.to_datetime(merged_data[col], errors = 'coerce')

In [93]:
merged_data['Frequency'].value_counts()

Frequency
Weekly     4202
Daily      4200
Monthly    4081
Name: count, dtype: int64

In [94]:
merged_data['customer_segment'].value_counts()

customer_segment
need_attention    4350
loyal             3927
at_risk           2388
premium           1590
Name: count, dtype: int64

In [95]:
nominal_columns = ['Gender', 'TotalInteractionType', 'Frequency', 'customer_segment'] # One hot
high_cardinal = ['Location', 'ProductList'] # mean encoding
ordinal_columns = ['Segment', 'Plan'] # label encoding

In [96]:
ohe = OneHotEncoder(sparse_output=False)

encode_data = merged_data.copy()

encoded_columns = []

for col in nominal_columns:
    transformed_col = ohe.fit_transform(encode_data[[col]])
    transformed_df = pd.DataFrame(transformed_col, columns = [f"{col}_{cat}" for cat in ohe.categories_[0]])
    
    encoded_columns.append(transformed_df)
    
encode_data = pd.concat([encode_data] + encoded_columns, axis = 1)
encode_data.drop(columns = nominal_columns, inplace = True)

In [97]:
le = LabelEncoder()

for col in ordinal_columns:
    encode_data[col] = le.fit_transform(encode_data[col])

In [98]:
for col in high_cardinal:
    encode_data[col] = encode_data.groupby(col)['TotalPurchaseValue'].transform('mean')

In [99]:
encode_data.drop(columns = 'customer_segment_nan', inplace = True)

In [100]:
encode_data

,CustomerID,Name,Age,Location,Email,Phone,Address,Segment,NPS,ChurnLabel,Timestamp,TotalPurchaseFrequency,TotalPurchaseValue,ProductList,Plan,Start_Date,End_Date,numEmails,numCalls,numChats,FirstInteractionDate,LastInteractionDate,AVGLatePayment,NumPaymentMethods,PageViews,TimeSpent(minutes),ActionCount,unique_pages,most_recent_action_date,Logins,Rating,Comment,AVGOpenDays,AVGClickDays,Gender_Female,Gender_Male,TotalInteractionType_Call,TotalInteractionType_Call|Chat,TotalInteractionType_Call|Chat|Email,TotalInteractionType_Call|Email,TotalInteractionType_Chat,TotalInteractionType_Chat|Email,TotalInteractionType_Email,Frequency_Daily,Frequency_Monthly,Frequency_Weekly,customer_segment_at_risk,customer_segment_loyal,customer_segment_need_attention,customer_segment_premium
0,1001,Mark Barrett,31,3155.595000,allison74@example.net,3192528777,"61234 Shelley Heights Suite 467\nCohentown, GU...",1,3,1,2020-01-27 01:36:49,38,3994.72,3994.720000,6,2020-06-08,2022-10-27,1,1,2,2019-09-26,2021-07-25,13.333333,3,49,15,24,13,2022-11-07 02:24:31,19,1,I move baby go small big. Office institution s...,818.0,319.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1002,Jeremy Welch,66,1477.077500,fmiller@example.com,231-587-1818x8651,"4959 Jennifer Junction\nNew Angelaport, TN 87397",2,6,0,2019-01-06 18:30:03,4,2844.35,2844.350000,12,2021-07-21,2022-05-07,10,5,4,2019-01-12,2022-12-13,3.333333,3,100,9,24,13,2022-12-05 00:27:08,9,2,Wish what bag cut life. Statement might opport...,110.0,88.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1003,Brandon Patel,36,1866.520000,jasonbrown@example.org,(270)633-9095,"38701 Amanda Brook Apt. 076\nKimshire, NJ 62516",1,3,0,2019-04-30 04:25:10,14,1866.52,1866.520000,5,2019-10-05,2020-08-19,1,1,1,2019-10-09,2022-01-04,2.666667,3,1,97,12,7,2022-11-02 15:59:43,19,4,Some Democrat guess but short. Whether behind ...,333.0,117.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1004,Tina Martin,62,2473.600000,matthew62@example.net,050.081.8706x11982,"67324 Ashley Coves\nSouth John, RI 29650",2,1,1,2020-03-03 17:33:28,28,1378.64,1378.640000,15,2020-01-14,2022-03-27,18,17,24,2019-01-03,2022-11-10,26.333333,3,25,31,47,14,2022-12-08 18:10:18,4,1,Yard feel never miss ask billion Congress. Fly...,159.0,423.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1005,Christopher Rodriguez,68,3795.523750,shannonstrickland@example.org,+1-701-854-4915x724,"01169 Miller Mission\nWest Anthonyburgh, WY 47359",2,3,0,2019-04-05 22:42:22,39,2425.05,2425.050000,0,2021-04-08,2022-11-09,5,4,1,2019-04-10,2022-12-19,0.666667,3,77,51,30,12,2022-12-30 23:37:00,12,3,Ten determine unit interview challenge stock. ...,193.0,96.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,13479,Catherine Lee,55,1196.560000,kgarcia@example.com,(977)150-1569x426,"836 Gonzalez Drive Apt. 334\nLake Kelly, RI 01667",0,8,0,2022-01-27 00:27:28,14,1196.56,1196.560000,5,2019-06-15,2021-06-29,3,3,4,2019-05-07,2022-10-09,1.000000,3,70,57,6,6,2022-10-02 14:18:14,22,2,Light appear fight lawyer where star.,126.0,443.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
12479,13480,Kimberly Johnson,29,710.570000,carrjennifer@example.org,485.014.7824x5235,3715 Wiggins Harbors Suite 139\nEast Gregorybo...,0,7,0,2020-06-14 23:04:47,1,710.57,523.136667,8,2022-12-10,2022-12-28,0,1,2,2019-12-11,2022-11-05,2.000000,3,71,66,9,8,2022-11-10 14:12:37,25,3,Yet very girl history. Thing late dream you re...,167.0,40.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
12480,13481,Lisa Rodgers,38,5154.420000,william37@example.com,951-489-9554,"56050 Steven Summit Suite 383\nJohnmouth, NM 2...",2,1,1,2020-10-13 15:09:13,63,5154.42,5154.420000,2,2021-07-04,2021-07-24,11,9,6,201

In [101]:
encode_data.dtypes

CustomerID                                       int64
Name                                            object
Age                                              int64
Location                                       float64
Email                                           object
Phone                                           object
Address                                         object
Segment                                          int64
NPS                                              int64
ChurnLabel                                       int64
Timestamp                               datetime64[ns]
TotalPurchaseFrequency                           int64
TotalPurchaseValue                             float64
ProductList                                    float64
Plan                                             int64
Start_Date                              datetime64[ns]
End_Date                                datetime64[ns]
numEmails                                        int64
numCalls  

In [102]:
date_col_to_engineer = encode_data.select_dtypes(include = 'datetime64').columns.tolist()
for col in date_col_to_engineer:
    encode_data[col + '_Year'] = encode_data[col].dt.year
    encode_data[col + '_Month'] = encode_data[col].dt.month
    encode_data[col + '_Day'] = encode_data[col].dt.day
    
    encode_data.drop(columns = [col], inplace = True)

In [103]:
pd.set_option('display.max_columns', None)

In [104]:
encode_data

,CustomerID,Name,Age,Location,Email,Phone,Address,Segment,NPS,ChurnLabel,TotalPurchaseFrequency,TotalPurchaseValue,ProductList,Plan,numEmails,numCalls,numChats,AVGLatePayment,NumPaymentMethods,PageViews,TimeSpent(minutes),ActionCount,unique_pages,Logins,Rating,Comment,AVGOpenDays,AVGClickDays,Gender_Female,Gender_Male,TotalInteractionType_Call,TotalInteractionType_Call|Chat,TotalInteractionType_Call|Chat|Email,TotalInteractionType_Call|Email,TotalInteractionType_Chat,TotalInteractionType_Chat|Email,TotalInteractionType_Email,Frequency_Daily,Frequency_Monthly,Frequency_Weekly,customer_segment_at_risk,customer_segment_loyal,customer_segment_need_attention,customer_segment_premium,Timestamp_Year,Timestamp_Month,Timestamp_Day,Start_Date_Year,Start_Date_Month,Start_Date_Day,End_Date_Year,End_Date_Month,End_Date_Day,FirstInteractionDate_Year,FirstInteractionDate_Month,FirstInteractionDate_Day,LastInteractionDate_Year,LastInteractionDate_Month,LastInteractionDate_Day,most_recent_action_date_Year,most_recent_action_date_Month,most_recent_action_date_Day
0,1001,Mark Barrett,31,3155.595000,allison74@example.net,3192528777,"61234 Shelley Heights Suite 467\nCohentown, GU...",1,3,1,38,3994.72,3994.720000,6,1,1,2,13.333333,3,49,15,24,13,19,1,I move baby go small big. Office institution s...,818.0,319.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2020,1,27,2020,6,8,2022,10,27,2019,9,26,2021,7,25,2022,11,7
1,1002,Jeremy Welch,66,1477.077500,fmiller@example.com,231-587-1818x8651,"4959 Jennifer Junction\nNew Angelaport, TN 87397",2,6,0,4,2844.35,2844.350000,12,10,5,4,3.333333,3,100,9,24,13,9,2,Wish what bag cut life. Statement might opport...,110.0,88.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2019,1,6,2021,7,21,2022,5,7,2019,1,12,2022,12,13,2022,12,5
2,1003,Brandon Patel,36,1866.520000,jasonbrown@example.org,(270)633-9095,"38701 Amanda Brook Apt. 076\nKimshire, NJ 62516",1,3,0,14,1866.52,1866.520000,5,1,1,1,2.666667,3,1,97,12,7,19,4,Some Democrat guess but short. Whether behind ...,333.0,117.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,2019,4,30,2019,10,5,2020,8,19,2019,10,9,2022,1,4,2022,11,2
3,1004,Tina Martin,62,2473.600000,matthew62@example.net,050.081.8706x11982,"67324 Ashley Coves\nSouth John, RI 29650",2,1,1,28,1378.64,1378.640000,15,18,17,24,26.333333,3,25,31,47,14,4,1,Yard feel never miss ask billion Congress. Fly...,159.0,423.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2020,3,3,2020,1,14,2022,3,27,2019,1,3,2022,11,10,2022,12,8
4,1005,Christopher Rodriguez,68,3795.523750,shannonstrickland@example.org,+1-701-854-4915x724,"01169 Miller Mission\nWest Anthonyburgh, WY 47359",2,3,0,39,2425.05,2425.050000,0,5,4,1,0.666667,3,77,51,30,12,12,3,Ten determine unit interview challenge stock. ...,193.0,96.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,2019,4,5,2021,4,8,2022,11,9,2019,4,10,2022,12,19,2022,12,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12478,13479,Catherine Lee,55,1196.560000,kgarcia@example.com,(977)150-1569x426,"836 Gonzalez Drive Apt. 334\nLake Kelly, RI 01667",0,8,0,14,1196.56,1196.560000,5,3,3,4,1.000000,3,70,57,6,6,22,2,Light appear fight lawyer where star.,126.0,443.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2022,1,27,2019,6,15,2021,6,29,2019,5,7,2022,10,9,2022,10,2
12479,13480,Kimberly Johnson,29,710.570000,carrjennifer@example.org,485.014.7824x5235,3715 Wiggins Harbors Suite 139\nEast Gregorybo...,0,7,0,1,710.57,523.136667,8,0,1,2,2.000000,3,71,66,9,8,25,3,Yet very girl history. Thing late dream you re...,167.0,40.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2020,6,14,2022,12,10,2022,12,28,2019,12,11,2022,11,5,2022,11,10
12480,13481,Lisa Rodgers,38,5154.420000,william37@

In [105]:
encode_data.drop(columns = 'CustomerID', inplace=True)

In [106]:
#encode_data.to_csv('clean_train_data', index = False)

In [107]:
#####################################

In [123]:
class ChurnPreprocessor:
    def __init__(self):
        self.ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.label_encoders = {}
        self.high_card_cols = ['Location', 'ProductList']
        self.nominal_cols = ['Gender', 'TotalInteractionType', 'Frequency', 'customer_segment']
        self.ordinal_cols = ['Segment', 'Plan']

    def fit(self, df):
        # One-hot encoding
        self.ohe.fit(df[self.nominal_cols])
        # Label encoding
        for col in self.ordinal_cols:
            le = LabelEncoder()
            le.fit(df[col])
            self.label_encoders[col] = le
        # High-card mean map
        self.high_card_meanmaps = {
            col: df[[col, 'TotalPurchaseValue']].groupby(col)['TotalPurchaseValue'].mean().to_dict()
            for col in self.high_card_cols
        }
        return self
        
    def transform(self, df):
        df = df.copy()
        ohe_df = pd.DataFrame(
            self.ohe.transform(df[self.nominal_cols]),
            columns=self.ohe.get_feature_names_out(self.nominal_cols),
            index=df.index
        )
        df.drop(columns=self.nominal_cols, inplace=True)
        for col, le in self.label_encoders.items():
            df[col] = le.transform(df[col])
        for col in self.high_card_cols:
            df[col] = df[col].map(self.high_card_meanmaps[col])
        df = pd.concat([df, ohe_df], axis=1)
        return df

In [124]:
import joblib

prep = ChurnPreprocessor().fit(merged_data)
joblib.dump(prep, 'churn_preprocessor.pkl')

['churn_preprocessor.pkl']